## Session 14

For this project you have 4 files containing information about persons.


In [9]:
### Define schema for each CSV
from collections import namedtuple
from itertools import islice
from datetime import datetime
import csv
import pandas as pd


employement_tuple = namedtuple("Employment_Details",['employer','department','employee_id','ssn'])
personal_info_tuple = namedtuple("Personal_Info",['ssn','first_name','last_name','gender','language'])
update_status_tuple = namedtuple("Update_Status",['ssn','last_updated','created'])
vehicles_tuple = namedtuple("Vehicle_Details",['ssn','vehicle_make','vehicle_model','model_year'])
all_details_tuple = namedtuple("Full_Details",['ssn','fname', 'lname', 'gender', 'lang','employer', 'dept', 'emp_id','last_upd', 'created','vmake','vmodel','model_year'])
datetime_fields = {'model_year':"%Y", "last_updated":"%Y-%m-%dT%H:%M:%SZ", "created":"%Y-%m-%dT%H:%M:%SZ" }



In [2]:
cd "/mnt/c/Users/rajy/OneDrive - Nokia/EVA4P2/theseer"

/mnt/c/Users/rajy/OneDrive - Nokia/EVA4P2/theseer


In [3]:
from datetime import datetime

def convert_datatypes(splitted_list, tuple_type):
    '''
    Converts a splitted array into respective namedtuple type.
    We have only 3 non-string columns i.e last_updated, created and vehicle's model_year
    '''
#     print(splitted_list, tuple_type)
    if len(tuple_type._fields) != len(splitted_list):
        print(f"Input list doesn't match the tuple type {tuple_type}")
        raise TypeError

    return  (datetime.strptime(val, datetime_fields[t_type]) if(t_type in datetime_fields) else str(val) for val,t_type in zip(splitted_list, tuple_type._fields))


def lazy_file_reader(file_name, mode='r',skip_header=True):
    '''
    A generator function to read and yield one line from a file indicated by "file_name"
    '''
    
    with open(file_name, mode, encoding='utf8') as f:
        if skip_header:
            next(f)
#         for line in f:
        try:
            yield from csv.reader(f, delimiter=',', quotechar='"')
        except Exception as e:
            print("Parsing Exception at:",e)

def file_specific_iterator(generator_type, tuple_type, max_count=None):
    '''
    Creates a slice of a given generator type and yields namedtuple based on tuple_type.
    '''
    for row in islice(generator_type, max_count):
        try:
            yield tuple_type(*convert_datatypes(row, tuple_type))
        except ValueError:
            print("Error for:",row)
            pass
 


In [4]:
class FullDbCreator:
    '''
    Iterable that takes the filenames as input and creates generators from each file 
    and returns one row of the merged file
    '''
    def __init__(self, file_name_list, max_count=None, max_recorded_date=None):
        '''
        Init function to create class object.
        '''
        self.__file_name_list = file_name_list
        self.__max_count = max_count
        try:
            if max_recorded_date is not None:
                self.__max_recorded_date = datetime.strptime(max_recorded_date,'%d/%m/%Y')
            else:
                self.__max_recorded_date = None
        except Exception as e:
            print('Invalid date format')
            raise e           
    
    def __iter__(self):
        '''
        Iterator to call static generator function that aggregates and creates the final representation format.
        '''
        return FullDbCreator.file_specific_iterator(self.__file_name_list, self.__max_count, self.__max_recorded_date)

    @staticmethod
    def file_specific_iterator(file_name_list, max_count=None, max_recorded_date=None):
        '''
        Adapter styled function to invoke individual iterators for each file type.
        The iterators are invoked only once, however the yield is called in loop till StopIteration.
        After extracting the values from each file's iterator, we do the following:
        1. Check if SSN's match across iterators.
        2. If max_recorded_date is set then only such records whose last_updated date is more than max_recorded_date are selected 
        '''
        vehicle_val = file_specific_iterator(lazy_file_reader(file_name_list[0]), vehicles_tuple, max_count)
        update_val = file_specific_iterator(lazy_file_reader(file_name_list[1]), update_status_tuple, max_count)
        emp_val = file_specific_iterator(lazy_file_reader(file_name_list[2]), employement_tuple, max_count)
        personal_val = file_specific_iterator(lazy_file_reader(file_name_list[3]), personal_info_tuple, max_count)
        while True:
            try:

                ssn,fname, lname, gender, lang,employer, dept, emp_id, emp_ssn, pers_ssn,last_upd, created, v_ssn,vmake,vmodel,model_year = *next(personal_val),*next(emp_val),*next(update_val),*next(vehicle_val)
                if(len(set([ssn,emp_ssn,pers_ssn,v_ssn])) > 1 ):
                    print("Mismatched SSN", ssn,emp_ssn,pers_ssn,v_ssn)
                    raise StopIteration
                if (max_recorded_date is not None and last_upd < max_recorded_date):
                    continue                    
                yield all_details_tuple(ssn,fname, lname, gender, lang,employer, dept, emp_id, last_upd, created,vmake,vmodel,model_year)
            except StopIteration:
                break

    
def get_most_pref_carmake_genderwise(full_table: 'List'):
    '''
    Returns the most popular models for each gender.
    Algorithm:
    For each gender type:
    1. Sort the gender specific entries based on car's make
    2. Create a map of Make vs Count
    3. Sort the map based on count in descending order
    4. Select all the models that have highest score.
    '''
    result_dict = {}
    for gender_val in set(map(lambda x: x.gender, full_table)):

        genderwise_list = sorted(list(filter(lambda x: x.gender == gender_val, full_table)), key=lambda x:x.vmake)
        model_list = [x.vmake for x in genderwise_list]

        x = { i: model_list.count(i) for i in set(model_list)}
        y = list(sorted(x.items(), key=lambda item: item[1], reverse=True))
        selected_list = list(filter(lambda x: x[1] >= y[0][1], y))
        result_dict[gender_val] =selected_list# (y[0], y[1])
    return result_dict

### Goal 1
Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

For now these four iterators are just separate, independent iterators.

In [19]:
vehicles_gen = file_specific_iterator(lazy_file_reader('./EPAI3/TSAI-EPAi-30/Session 14/Assignment/vehicles.csv'), vehicles_tuple)
update_status_gen = file_specific_iterator(lazy_file_reader('./EPAI3/TSAI-EPAi-30/Session 14/Assignment/update_status.csv'), update_status_tuple)
employement_gen = file_specific_iterator(lazy_file_reader('./EPAI3/TSAI-EPAi-30/Session 14/Assignment/employment.csv'), employement_tuple)
personal_info_gen = file_specific_iterator(lazy_file_reader('./EPAI3/TSAI-EPAi-30/Session 14/Assignment/personal_info.csv'), personal_info_tuple)


In [16]:
pd.DataFrame(vehicles_gen).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ssn            1000 non-null   object        
 1   vehicle_make   1000 non-null   object        
 2   vehicle_model  1000 non-null   object        
 3   model_year     1000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 31.4+ KB


In [20]:
pd.DataFrame(update_status_gen).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ssn           1000 non-null   object        
 1   last_updated  1000 non-null   datetime64[ns]
 2   created       1000 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 23.6+ KB


## Goal 2
Create a single iterable that combines all the columns from all the iterators. 
The iterable should yield named tuples containing all the columns. Make sure that the SSN's across the files match!
All the files are guaranteed to be in SSN sort order, and every SSN is unique, and every SSN appears in every file.

Make sure the SSN is not repeated 4 times - one time per row is enough!


In [ ]:
class FullDbCreator:
    '''
    Iterable that takes the filenames as input and creates generators from each file 
    and returns one row of the merged file
    '''
    def __init__(self, file_name_list, max_count=None):
        self.__vehicles_gen = file_specific_iterator(lazy_file_reader(file_name_list[0]), vehicles_tuple, max_count)
        self.__update_status_gen = file_specific_iterator(lazy_file_reader(file_name_list[1]), update_status_tuple,max_count)
        self.__employement_gen = file_specific_iterator(lazy_file_reader(file_name_list[2]), employement_tuple,max_count)
        self.__personal_info_gen = file_specific_iterator(lazy_file_reader(file_name_list[3]), personal_info_tuple,max_count)
        self.__max_count = max_count
    
    def __iter__(self):
#         return self.DbIterator(self, self.__max_count)
        return self.DbIterator([self.__vehicles_gen, self.__update_status_gen, self.__employement_gen, self.__personal_info_gen], self.__max_count)
    
    class DbIterator:
        def __init__(self, gen_list, max_count=None):
            self._index = 0
#             self.__gen_list = gen_list
            self.__vehicles_iter = gen_list[0]
            self.__update_status_iter = gen_list[1]
            self.__employement_iter = gen_list[2]
            self.__personal_info_iter = gen_list[3]
        def __iter__(self):
            '''
            Basic __iter__ method that returns an instance of the iterator
            '''
#             print("Calling PolygonSqIterator instance __iter__")
            return self
        
        def __next__(self):
            '''
            Implementation of __next__ function. Throws StopIteration when length is bypassed.
            '''
            try:
#                 return (*self.__gen_list.__vehicles_gen,\
#                         *self.__gen_list.__update_status_gen,\
#                         *self.__gen_list.__employement_gen,\
#                         *self.__gen_list.__personal_info_gen)
                return (next(self.__vehicles_iter), next(self.__update_status_iter),next(self.__employement_iter),next(self.__personal_info_iter))
            except Exception as e:
                print("Exception during iteration",e)
                raise StopIteration

In [21]:
file_list = [ './EPAI3/TSAI-EPAi-30/Session 14/Assignment/vehicles.csv',
             './EPAI3/TSAI-EPAi-30/Session 14/Assignment/update_status.csv',
    './EPAI3/TSAI-EPAi-30/Session 14/Assignment/employment.csv',
 './EPAI3/TSAI-EPAi-30/Session 14/Assignment/personal_info.csv'
]

# glob.glob('./EPAI3/TSAI-EPAi-30/Session 14/Assignment/*csv')

my_full_class = FullDbCreator(file_list)#, max_recorded_date='20/3/2018')
iter_one = iter(my_full_class)

pd.DataFrame(my_full_class)

,ssn,fname,lname,gender,lang,employer,dept,emp_id,last_upd,created,vmake,vmodel,model_year
0,100-53-9824,Sebastiano,Tester,Male,Icelandic,Stiedemann-Bailey,Research and Development,29-0890771,2017-10-07 00:14:42,2016-01-24 21:19:30,Oldsmobile,Bravada,1993-01-01
1,101-71-4702,Cayla,MacDonagh,Female,Lao,Nicolas and Sons,Sales,41-6841359,2017-01-23 11:23:17,2016-01-27 04:32:57,Ford,Mustang,1997-01-01
2,101-84-0356,Nomi,Lipprose,Female,Yiddish,Connelly Group,Research and Development,98-7952860,2017-10-04 11:21:30,2016-09-21 23:04:07,GMC,Yukon,2005-01-01
3,104-22-0928,Justinian,Kunzelmann,Male,Dhivehi,Upton LLC,Marketing,56-9817552,2017-03-28 12:38:29,2016-04-15 11:37:17,Oldsmobile,Intrigue,2000-01-01
4,104-84-7144,Claudianus,Brixey,Male,Afrikaans,Zemlak-Olson,Business Development,46-2886707,2018-02-19 01:34:33,2016-03-15 14:07:57,Ford,Crown Victoria,2008-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,898-86-8860,Hershel,Few,Male,Gagauz,Olson-Johnson,Support,36-7761827,2017-07-03 14:27:26,2016-11-23 23:34:17,Lotus,Esprit,2003-01-01
996,899-09-1766,Joe,Myderscough,Male,Filipino,Cronin LLC,Services,83-8730953,2017-07-31 02:26:57,2016-10-27 18:41:56,Toyota,Camry,2012-01-01
997,899-39-7179,Rani,Naisbitt,Female,Greek,Little-Hansen,Business Development,45-8154226,2017-08-02 13:32:33,2016-07-27 12:47:05,Infiniti,FX,2006-01-01
998,899-39-7378,Lindsay,Colledge,Female,Croatian,"Christiansen, Ondricka and O'Connell",Marketing,32-7034344,2017-03-27 19:31:32,2016-09-04 15:09:22,Toyota,Land Cruiser,1999-01-01


## Goal 3
Next, you want to identify any stale records, where stale simply means the record has not been updated since 3/1/2017 (e.g. last update date < 3/1/2017). Create an iterator that only contains current records (i.e. not stale) based on the `last_updated` field from the `status_update` file.

In [23]:
file_list = [ './EPAI3/TSAI-EPAi-30/Session 14/Assignment/vehicles.csv',
             './EPAI3/TSAI-EPAi-30/Session 14/Assignment/update_status.csv',
    './EPAI3/TSAI-EPAi-30/Session 14/Assignment/employment.csv',
 './EPAI3/TSAI-EPAi-30/Session 14/Assignment/personal_info.csv'
]

# glob.glob('./EPAI3/TSAI-EPAi-30/Session 14/Assignment/*csv')

my_full_class = FullDbCreator(file_list, max_recorded_date='20/3/2017')
iter_one = iter(my_full_class)

pd.DataFrame(my_full_class)

,ssn,fname,lname,gender,lang,employer,dept,emp_id,last_upd,created,vmake,vmodel,model_year
0,100-53-9824,Sebastiano,Tester,Male,Icelandic,Stiedemann-Bailey,Research and Development,29-0890771,2017-10-07 00:14:42,2016-01-24 21:19:30,Oldsmobile,Bravada,1993-01-01
1,101-84-0356,Nomi,Lipprose,Female,Yiddish,Connelly Group,Research and Development,98-7952860,2017-10-04 11:21:30,2016-09-21 23:04:07,GMC,Yukon,2005-01-01
2,104-22-0928,Justinian,Kunzelmann,Male,Dhivehi,Upton LLC,Marketing,56-9817552,2017-03-28 12:38:29,2016-04-15 11:37:17,Oldsmobile,Intrigue,2000-01-01
3,104-84-7144,Claudianus,Brixey,Male,Afrikaans,Zemlak-Olson,Business Development,46-2886707,2018-02-19 01:34:33,2016-03-15 14:07:57,Ford,Crown Victoria,2008-01-01
4,105-27-5541,Federico,Aggett,Male,Chinese,"Kohler, Bradtke and Davis",Support,80-0975518,2017-07-24 08:58:52,2016-07-23 17:58:35,Ford,Mustang,2001-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,898-86-8860,Hershel,Few,Male,Gagauz,Olson-Johnson,Support,36-7761827,2017-07-03 14:27:26,2016-11-23 23:34:17,Lotus,Esprit,2003-01-01
826,899-09-1766,Joe,Myderscough,Male,Filipino,Cronin LLC,Services,83-8730953,2017-07-31 02:26:57,2016-10-27 18:41:56,Toyota,Camry,2012-01-01
827,899-39-7179,Rani,Naisbitt,Female,Greek,Little-Hansen,Business Development,45-8154226,2017-08-02 13:32:33,2016-07-27 12:47:05,Infiniti,FX,2006-01-01
828,899-39-7378,Lindsay,Colledge,Female,Croatian,"Christiansen, Ondricka and O'Connell",Marketing,32-7034344,2017-03-27 19:31:32,2016-09-04 15:09:22,Toyota,Land Cruiser,1999-01-01


## Goal 4
Find the largest group of car makes for each gender. 
Possibly more than one such group per gender exists (equal sizes).

In [24]:
file_list = [ './EPAI3/TSAI-EPAi-30/Session 14/Assignment/vehicles.csv',
             './EPAI3/TSAI-EPAi-30/Session 14/Assignment/update_status.csv',
    './EPAI3/TSAI-EPAi-30/Session 14/Assignment/employment.csv',
 './EPAI3/TSAI-EPAi-30/Session 14/Assignment/personal_info.csv'
]

my_full_list = FullDbCreator(file_list)
get_most_pref_carmake_genderwise(my_full_list)

{'Female': [('Chevrolet', 48), ('Ford', 48)], 'Male': [('Ford', 44)]}